# How to acquire radial velocities with "astroquery"

This notebook sets out some basic routines to search for literature radial velocity values in the VizieR database, using the astropy routine "astroquery".

In this specific example I have a set of targets from the Wide Field Infrared Survey (WISE), where the source identifiers implicitly define the celestial coordinates (RA/DEC). This program uses a simple string extraction to create RA and DEC arrays, which are used as positional arguments for the Vizier query.

A Vizier query is performed within 5 arcseconds on all catalogues that contain the keyword "stars", and a string match is made for a set of possible column names that would describe the radial velocity (RV).

The first table contains the WISE identifier, the catalogue from Vizier, the angular distance of the matched Vizier target and the RV with an error (eRV).

Steps are made to remove duplicate entries, substitute entries without RV errors with a dummy "-999" and remove entries where there are no RV data.

The final step is to remove entries that appear to be duplicate entries from "child" catalogues that contain adopted RVs from a parent sample. This is important to minimise biases when attempting to calculate a final RV value based on multiple individual RV measurements. An example of this might be a set of new RV catalogues, all of which use the Gaia DR2 value, where we only want to keep the original DR2 measurement. This was done by identifying targets with very similar RVs and errors, and simply keeping the measurement with the lowest number of significant figures (I found that a lot of "child" catalogues give a very large number of significant figures, presumably from numerical rounding).

### step (1):
Import the necessary modules

In [6]:
from os.path import exists

import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier
from astropy.io import ascii
from astropy.table import unique, Table
from collections import Counter
from time import sleep

### step (2)
Read in the list of WISE identifiers

In [11]:
file_name = input('What is the name of your file?\n')

file_exists = exists(file_name)
if file_exists:
    file_in = ascii.read(file_name)
#    name_coo  = file_in['name']
    print("Great! Let's get some data!")
else:
    print("file doesn't exist, try again!")

What is the name of your file?
../AO23/Volans/LDB/Volans_Input.csv
Great! Let's get some data!


### step (3)
Read each row into a for loop, break up the string into HMS and DMS, concatenate and store into "ra" and "dec" arrays - create a SkyCoord structure.

In [14]:
#ra        = [name_coo[i][1:3]+'h'+name_coo[i][3:5]+'m'+name_coo[i][5:10]+'s' for i in range(len(name_coo))]
#dec       = [name_coo[i][10:13]+'d'+name_coo[i][13:15]+'m'+name_coo[i][15:19]+'s' for i in range(len(name_coo))]
#print(file_in)
name_coo = file_in["Designation"]#.astype(str)
c = SkyCoord(file_in["Radeg_1"], file_in["Decdeg_1"], frame='icrs', unit='deg')


### step (4)
Make a string array of possible names that might refer to RVs.\
Inevitably some will be missed out but this is the best we can do with the Vizier nomenclature.

In [15]:
rv_names = ['RV', 'VR', 'HRV', 'Vrad', 'RV_final', 'RVf', 'RVfin', 'RV_f', 'RV_fin',  
            'eRV', 'ERV', 'E_RV', 'e_RV', 'e_HRV', 'eVrad', 'e_VR']

### step (5)
Query Vizier Catalogs that have the astronomy keyword "stars".
There is also a feature called "ucd" in Simbad that ensures the quantity is the actual RV measurement and not a predicted/kinematic RV -- I don't use it here, but maybe it'll be useful in the future.

In [16]:
v = Vizier(columns=['+_r']+rv_names, keywords=["Stars"])

### step (6)
Executing the Vizier query for each target in the list.\
\
a) make an Astropy table to store the results.\
b) perform a query within 5 arcseconds.\
c) store the results and make a list of Vizier table names.\
d) test if a given Vizier table contains one of our RV column identifier names.\
e) if satisfied, extract the RV and error columns, and replace missing entries with "-999"\
f) store results to the Astropy table.

In [17]:
t = Table(names=('Name', 'VizCat', 'd_as', 'RV', 'eRV'), dtype=('str', 'str', 'float', 'float', 'float'))
print(t)
for j in range(len(c)):
    # search within x-arcsec (x=5)
    result = v.query_region(c[j], width="5s")
    # Loose method to take the name of the Vizier catalog
    # and just extract the Vizier table code (Good for
    # referring to later on!)
    table_names = str(result)
    l = table_names.split("'")[1::2]
    # Check each Vizier table that turned up in the
    # cross-match, only deal with the ones that actually
    # have some RV quantities.
    for num, table in enumerate(result):
        # make boolean array for the rv_names that returns
        # True if matched with one of the Vizier table
        # columns
        res = np.isin(rv_names, table.colnames)
        if np.any(res):
            # Return the RV values
            name_m = np.array(rv_names)[np.array(res)]
            for name_x in name_m:
                x = [table[name_x][0] for name_x in name_m]
                if len(x) == 2:
                    if type(x[0]) == np.ma.core.MaskedConstant:
                        x[0] = -999
                    if type(x[1]) == np.ma.core.MaskedConstant:
                        x[1] = -999
                if len(x) == 1:
                    x.append(-999)
                    if type(x[0]) == np.ma.core.MaskedConstant:
                        x[0] = -999
                print(name_coo[j], l[num].split(":")[1], table['_r'][0], x[0], x[1])
                t.add_row((name_coo[j], l[num].split(":")[1], table['_r'][0], x[0], x[1]))

Name VizCat d_as  RV eRV
---- ------ ---- --- ---
HD 80563 I/345/gaia2 0.05 23.75 0.42
HD 80563 I/345/gaia2 0.05 23.75 0.42
HD 80563 I/355/gaiadr3 0.076 24.23 0.21
HD 80563 I/355/gaiadr3 0.076 24.23 0.21
HD 80563 V/137D/XHIP 1.195 -999 -999
HD 80563 V/137D/XHIP 1.195 -999 -999
HD 80563 V/145/sky2kv5 0.74 -999 -999
HD 80563 J/A+A/530/A138/catalog 0.2 -999 -999
HD 80563 J/A+A/530/A138/catalog 0.2 -999 -999
HD 80563 J/A+A/546/A61/tablea1 0.744 -999 -999
HD 80563 J/A+A/546/A61/tablea1 0.744 -999 -999
HD 80563 J/A+A/623/A72/hipgpma 1.0 23.723 0.42
HD 80563 J/A+A/623/A72/hipgpma 1.0 23.723 0.42
HD 80563 J/A+A/649/A6/table1c 0.076 23.752 0.4158
HD 80563 J/A+A/649/A6/table1c 0.076 23.752 0.4158
HD 80563 J/A+A/657/A7/tablea1 0.75 23.7 0.42
HD 80563 J/A+A/657/A7/tablea1 0.75 23.7 0.42
HD 80563 J/A+A/657/A7/tablea2 0.076 23.7 0.42
HD 80563 J/A+A/657/A7/tablea2 0.076 23.7 0.42
HD 80563 J/A+A/657/A7/tablea3 0.076 23.77 0.42
HD 80563 J/A+A/657/A7/tablea3 0.076 23.77 0.42
HD 83946 I/345/gaia2 0.031 2

2MASS J08591527-6918426 I/345/gaia2 0.024 23.71 1.33
2MASS J08591527-6918426 I/345/gaia2 0.024 23.71 1.33
2MASS J08591527-6918426 I/355/gaiadr3 0.053 23.36 1.52
2MASS J08591527-6918426 I/355/gaiadr3 0.053 23.36 1.52
2MASS J08591527-6918426 J/A+A/649/A6/table1c 0.053 23.706 1.3318
2MASS J08591527-6918426 J/A+A/649/A6/table1c 0.053 23.706 1.3318
2MASS J08591527-6918426 J/A+A/657/A7/tablea3 0.053 23.69 1.33
2MASS J08591527-6918426 J/A+A/657/A7/tablea3 0.053 23.69 1.33
2MASS J09311626-6414270 I/345/gaia2 0.023 25.71 3.1
2MASS J09311626-6414270 I/345/gaia2 0.023 25.71 3.1
2MASS J09311626-6414270 I/355/gaiadr3 0.05 47.91 12.41
2MASS J09311626-6414270 I/355/gaiadr3 0.05 47.91 12.41
2MASS J09311626-6414270 J/A+A/649/A6/table1c 0.05 25.713 3.1008
2MASS J09311626-6414270 J/A+A/649/A6/table1c 0.05 25.713 3.1008
2MASS J09311626-6414270 J/A+A/657/A7/tablea3 0.05 25.69 3.1
2MASS J09311626-6414270 J/A+A/657/A7/tablea3 0.05 25.69 3.1
2MASS J09174971-6628140 I/345/gaia2 0.026 24.92 2.34
2MASS J09174971

ReadTimeout: HTTPConnectionPool(host='vizier.u-strasbg.fr', port=80): Read timed out. (read timeout=60)

### step (7)
Remove duplicate entries and entries with empty RV values.\
Replace targets with empty eRV with "-999".

In [28]:
l = list(t.colnames)
t = unique(t, keys=l, keep='last')
t.remove_rows(np.where(t['RV'] == -999)[0])
t.sort('Name')
print([x for x in t])

[<Row index=0>
        Name           VizCat     d_as     RV           eRV        
       str19           str27    float64 float64       float64      
------------------- ----------- ------- ------- -------------------
1005809361369063168 I/345/gaia2   0.097   21.41 0.38999998569488525, <Row index=1>
        Name            VizCat      d_as     RV           eRV        
       str19            str27     float64 float64       float64      
------------------- ------------- ------- ------- -------------------
1005809361369063168 I/355/gaiadr3   0.028   21.19 0.23999999463558197, <Row index=2>
        Name               VizCat          d_as     RV     eRV  
       str19               str27         float64 float64 float64
------------------- -------------------- ------- ------- -------
1005809361369063168 J/A+A/649/A6/table1c   0.028   21.41  0.3853, <Row index=3>
        Name               VizCat          d_as          RV                 eRV        
       str19               str27        

### step (8)
Quite often RV values quoted by catalogues are adopted RVs from some other work that actually performed the measurement. Perhaps because of rounding errors in computational methods, these values can show up with large numbers of significant figures. Browsing through some of the table above we see that the RV and eRV values are often very similar, and that usually the RV source catalogue (i.e., the catalogue from the actual measurement) provides a lower number of significant figures.\

In reality we only want to keep one measurement, not several similar derived values. This is to avoid biases when calculating a "final" RV from several individual measurements.\

To do this I make the following steps:\

a) Make a new table for the modified data to be stored into (makes life easier at the end).\
b) Group the data by name, and set up a for loop to read each group.\
c) Remove any entries whose RV and eRV values are identical.\
d) Sort the group in ascending eRV values.\
e) Create two square arrays, which are the differences in RV and eRV.\
f) Set a threshold criteria for "similar" RV and eRV values.\
g) Create a (square) boolean array for entries that satisfy the criteria.\
h) Make an algorithm which shows which targets have 2 or more similar RV (and eRV) values.\
i) Find the corresponding eRV value that provides the lowest number of significant figures.\
j) Store the values from (i) and print them to the new table.

In [40]:
t_gr = t.group_by('Name')
t_new = Table(names=('Name', 'VizCat', 'd_as', 'RV', 'eRV'), dtype=('str', 'str', 'float', 'float', 'float'))
# e_thr is the RV error value below which
# we are suspicious that if there are >1 measurements
# that these are just duplicates.
e_thr, r_thr = 0.02, 0.2
for t_g in t_gr.groups:
    t_g = unique(t_g, keys=l[-2:], keep='first')

    i_s = np.array(sorted(t_g['eRV']))
    i_d = np.argsort(t_g['eRV'])
    i_r = t_g['RV'][i_d]
    l_arr = []
    c = i_s[:, None] - i_s[None, :]
    r = i_r[:, None] - i_r[None, :]
    m_arr = (abs(c)<e_thr) & (abs(r)<r_thr)

    if len(i_s)>1:
        i, j = 0, 0
        while j < len(i_s)-1:
            print(len(i_s))
            j = j+1
            x = i
            if m_arr[i,j] == True:
                if j == len(i_s)-1:
                    arr = i_s[x:j+1].astype(str)
                    pos = x+np.where(arr == min(arr, key=len))[0][0]
                    print(arr, pos)
                    l_arr.append(pos)
            else:
                if j-x == 1:
                    print(i)
                    l_arr.append(i)
                    i=j
                else:
                    arr = i_s[x:j].astype(str)
                    pos = x+np.where(arr == min(arr, key=len))[0][0]
                    print(arr, pos)
                    l_arr.append(pos)
                    i=j
        print(' ')
        for k in l_arr:
            t_new.add_row(t_g[i_d[k]])
    elif len(i_s) == 1:
        t_new.add_row(t_g[i_d[0]])

4
0
4
4
['0.3853' '0.38999998569488525' '0.38999998569488525'] 1
 
7
0
7
7
['0.20000000298023224' '0.20000000298023224'] 1
7
7
7
['0.23000000417232513' '0.2326' '0.232607203109478'] 4
 
6
0
6
1
6
6
['0.3968' '0.4000000059604645'] 2
6
4
 
9
0
9
9
9
9
9
9
['0.6361' '0.63612611008832' '0.6399999856948853' '0.6399999856948853'
 '0.6399999856948853' '0.6399999856948853'] 1
9
7
 
5
0
5
5
5
['0.9081' '0.908104369472028' '0.9100000262260437' '0.9100000262260437'] 1
 
5
0
5
5
5
['0.717973712070449' '0.718' '0.7200000286102295' '0.7200000286102295'] 2
 
5
0
5
5
5
['0.4699999988079071' '0.4699999988079071' '0.473953271419243' '0.474'] 4
 
4
4
4
['0.358' '0.36000001430511475' '0.36000001430511475'] 0
 
10
0
10
1
10
2
10
3
10
10
['0.30000001192092896' '0.30000001192092896'] 4
10
6
10
10
['0.4000000059604645' '0.4000000059604645' '0.4034'] 9
 
7
0
7
1
7
7
7
['0.30000001192092896' '0.30000001192092896' '0.30000001192092896'] 2
7
5
 
4
0
4
4
['1.2599999904632568' '1.2599999904632568' '1.2629'] 3
 
4
4

4
0
4
4
['3.9878' '3.990000009536743' '3.990000009536743'] 1
 
16
0
16
1
16
2
16
16
16
16
['0.1899' '0.189920975829014' '0.1899999976158142' '0.1899999976158142'] 3
16
16
16
['0.4000000059604645' '0.4000000059604645' '0.4000000059604645'] 7
16
10
16
11
16
16
['0.8199999928474426' '0.82'] 13
16
14
 
2
0
 
5
0
5
5
5
['1.76899177472977' '1.769' '1.7699999809265137' '1.7699999809265137'] 2
 
5
5
5
5
['2.490000009536743' '2.490000009536743' '2.4927' '2.49272297947427'] 2
 
6
0
6
1
6
2
6
6
['1.8657' '1.8700000047683716' '1.8700000047683716'] 3
 
10
0
10
1
10
2
10
10
['1.1' '1.100000023841858'] 3
10
10
['1.100000023841858' '1.100000023841858'] 5
10
7
10
8
 
2
['0.4' '0.4000000059604645'] 0
 
3
3
['0.3' '0.30000001192092896'] 0
 
3
0
3
1
 
10
0
10
1
10
2
10
3
10
4
10
5
10
6
10
10
['5.800000190734863' '5.800000190734863'] 7
 
12
0
12
1
12
2
12
3
12
12
12
12
['0.5' '0.5' '0.5' '0.5'] 4
12
8
12
9
12
10
 
4
0
4
4
['0.3886' '0.38999998569488525' '0.38999998569488525'] 1
 
11
0
11
1
11
2
11
3
11
4
1

### step (9)
Enjoy your final clean RV table!\
Print to file if necessary...

In [41]:
print([t for t in t_new])
t_new.write('RV_values.csv', overwrite=True)

[<Row index=0>
        Name            VizCat      d_as     RV           eRV        
       str19            str27     float64 float64       float64      
------------------- ------------- ------- ------- -------------------
1005809361369063168 I/355/gaiadr3   0.028   21.19 0.23999999463558197, <Row index=1>
        Name               VizCat          d_as     RV     eRV  
       str19               str27         float64 float64 float64
------------------- -------------------- ------- ------- -------
1005809361369063168 J/A+A/649/A6/table1c   0.028   21.41  0.3853, <Row index=2>
       Name                VizCat           d_as           RV                  eRV        
      str19                str27          float64       float64              float64      
------------------ ---------------------- ------- -------------------- -------------------
101283987497967360 J/A+A/530/A138/catalog     3.0 -0.30000001192092896 0.20000000298023224, <Row index=3>
       Name               VizCat    